## Hard Encode Input Dictionary



In [1]:
import pandas as pd

In [2]:
input = {
    'zipcode': 99203,
    'family_members': 4,
    'monthly_income': 2000,
}

In [3]:
def framemaker(web_in):
# making dataframe out of dict  
  input_frame = pd.DataFrame(web_in, index=[0])

  input_frame = input_frame[['zipcode', 'family_members', 'monthly_income']]

  userinput = input_frame.iloc[[0]]

  return userinput 

In [4]:
user_input = framemaker(input)

In [5]:
user_input

,zipcode,family_members,monthly_income
0,99203,4,2000


## Make function that takes in dataframe, uses model, and can make a prediction

In [6]:
import pandas as pd
!pip install category_encoders==2.*
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from category_encoders import OneHotEncoder, OrdinalEncoder

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/fp-rap-build/data-science/main/data/hud_requirements%20-%20data.csv')

df

,zipcode,family_members,annual_income_requirements,qualifies
0,99203,1,43350,1
1,99203,2,49550,1
2,99203,3,55750,1
3,99203,4,61900,1
4,99203,5,66900,1
5,99203,6,71580,1
6,99203,7,76800,1
7,99203,8,81750,1
8,99203,1,43370,0
9,99203,2,49570,0


In [8]:
df['monthly_income'] = df['annual_income_requirements'].div(12).round(2)

In [9]:
def model_maker():
  train, test = train_test_split(df, train_size=0.80, test_size=0.20, 
                                 stratify=df['qualifies'], random_state=42)
  # select our target 
  target = 'qualifies'

  # Arrange data into X features matrix and y target vector
  features = ['zipcode', 'family_members', 'monthly_income']
  target = ['qualifies']

  X_train = train[features]
  y_train = train[target]
  X_test = test[features]
  y_test = test[target]

  lrmodel = Pipeline([
                  ('ohe', OneHotEncoder(use_cat_names=True)),
                  ('scaler', StandardScaler()),  
                  ('impute', SimpleImputer()),
                  ('classifier', LogisticRegressionCV())
                  ])
  lrmodel.fit(X_train, y_train)

  return lrmodel

lrmodel = model_maker()

def predict(user_input):

  if lrmodel.predict(user_input) == 1:
    predict = {'predict': 'User does not qualify.'}
    input.update(predict) 
    return input
  else:
    predict = {'predict':'User qualifies for rental assistance!'}
    input.update(predict)
    return input

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [11]:
print(predict(user_input))

{'zipcode': 99203, 'family_members': 4, 'monthly_income': 2000, 'predict': 'User qualifies for rental assistance!'}
